# Med2Vec

Required environment
- Python 2.7
- Required Packages: Theano, Pickle, Scikit-learn and Scipy

Import codes

run Med2Vec_Functions.ipynb

## Data

The format of the data required by the original package is a list, such as each sample is separated by a token [-1].

    Example:
    - Sample 1 - 2 visits: $[1,2,3], [4,5,6,7]$
    - Sample 2 - 3 visits: $[2,4], [8,3,1], [3]$
    - Final input: $[[1,2,3], [4,5,6,7], [-1], [2,4], [8,3,1], [3]]$

The data must be pickled.

In [ ]:
pickle.dump(..., open('data_pickle', 'wb'), -1)

## GridSearch

Optimized Parameters
- embDimSize : size of the intermediate representation
- hiddenDimSize : size of the final representation
- window size : size of the visit context window 

In [ ]:
embDimSize_list = [20, 50, 100]
hiddenDimSize_list = [20, 50, 100]
window_size_list = [1, 3, 5]

In [ ]:
N = # Number of unique codes
path = # path 

In [ ]:
File = {
    'seqFile' : path+'data_pickle',
    'labelFile' : '',
    'outFile' :path,
    'demoFile' : '',
    'numXcodes': N,
    'numYcodes': 0,
    'demoSize': 0,
    'batchSize' : 250, 
    'maxEpochs' : 5 ,
    'logEps': 1e-6,
    'L2_reg': 0.001
}

In [ ]:
df_gridsearch_word2vec = Gridsearch_Med2Vec(File, embDimSize_list, hiddenDimSize_list, window_size_list)

In [ ]:
print('Optimal hyperparameters are ')
print('Embedding_size : ' + str(df_gridsearch_word2vec.iloc[int(np.where(df_gridsearch_word2vec.Loss_test==np.min(df_gridsearch_word2vec['Loss_test']))[0].item())].embDimSize))
print('Hidden_size : ' + str(df_gridsearch_word2vec.iloc[int(np.where(df_gridsearch_word2vec.Loss_test==np.min(df_gridsearch_word2vec['Loss_test']))[0].item())].hiddenDimSize))
print('Window_size : ' + str(df_gridsearch_word2vec.iloc[int(np.where(df_gridsearch_word2vec.Loss_test==np.min(df_gridsearch_word2vec['Loss_test']))[0].item())].windowSize))

## Training

In [ ]:
embDimSize_opt = df_gridsearch_word2vec.iloc[int(np.where(df_gridsearch_word2vec.Loss_test==np.min(df_gridsearch_word2vec['Loss_test']))[0].item())].embDimSize
hiddenDimSize_opt = df_gridsearch_word2vec.iloc[int(np.where(df_gridsearch_word2vec.Loss_test==np.min(df_gridsearch_word2vec['Loss_test']))[0].item())].hiddenDimSize
windowSize_opt = df_gridsearch_word2vec.iloc[int(np.where(df_gridsearch_word2vec.Loss_test==np.min(df_gridsearch_word2vec['Loss_test']))[0].item())].windowSize

In [ ]:
File = {
    'seqFile' : path+'data_pickle',
    'labelFile' : '',
    'outFile' : path,
    'demoFile' : '',
    'numXcodes': N,
    'numYcodes': 0,
    'demoSize': 0,
    'embDimSize' : embDimSize_opt,
    'hiddenDimSize' : hiddenDimSize_opt,
    'windowSize' : windowSize_opt,
    'batchSize' : 250, 
    'maxEpochs' : 20,
    'logEps': 1e-6,
    'L2_reg': 0.001
}

In [ ]:
loss_train, loss_test = Train_Med2Vec(File)

## Resulting Patient Representation

patient_representation = []

for each unique patient :

    visit_embedding = np.zeros(hiddenDimSize_opt)

    for visit in the sequence of the patient :

        one_hot_encoded = np.zeros(File['numXcodes'])

        one_hot_encoded[visit] = 1

        code_embedding = np.maximum(one_hot_encoded.dot(model_param['W_emb']) + model_param['b_emb'],0)

        visit_embedding += np.maximum(code_embedding.dot(model_param['W_hidden']) + model_para['b_hidden'],0)
        
    patient_representation.append(visit_embedding)

In [ ]:
np.save('patient_representations', patient_representation)